In [1]:
import pandas as pd
import numpy as np
import os
import json
from collections import Counter
import string

In [20]:
all_df = pd.read_csv("../Pubmed_Bert/output/all_label.csv", index_col=0, header=0)

with open("../Pubmed_Search/search_results/search_results.txt","r") as f:
    spec_paper_dics = json.loads(f.read())

In [26]:
pdf_d = {}
for spec_type in spec_paper_dics.keys():
    
    pdf = pd.DataFrame(all_df.loc[all_df["物种类别"]==spec_type,"物种名称"].value_counts()) 
    pdf.columns = ["label_counts"]
    pdf["search_counts"] = 0
    pdf["portions"] = 0
    spdf = pd.DataFrame([spec_paper_dics[spec_type].keys(), [len(spec_paper_dics[spec_type][i]) for i in spec_paper_dics[spec_type].keys()]],index=["物种名称","Count"]).T
    spdf.index = spdf["物种名称"]
    
    for i in pdf.index:
        if (i in spdf.index) and (spdf.loc[i,"Count"]!=0):
            pdf.loc[i, "search_counts"] = spdf.loc[i, "Count"]
            pdf.loc[i, "portions"] = (pdf.loc[i, "label_counts"]/pdf.loc[i, "search_counts"]).round(4)
        else:
            pdf.loc[i, "search_counts"] = pdf.loc[i, "label_counts"]
            pdf.loc[i, "portions"] = (pdf.loc[i, "label_counts"]/pdf.loc[i, "search_counts"]).round(2)
    pdf_d[spec_type] = pdf
    # pdf.index = [i.capitalize() for i in pdf.index]
    pdf.to_csv("output/right_portion_%s.csv" % spec_type)


In [25]:
list(pdf.loc[pdf["label_counts"] >= 1000].index)

['Influenza a virus',
 'Respiratory syncytial virus',
 'Human immunodeficiency virus',
 'H1n1',
 'Human immunodeficiency virus 1',
 'Human immunodeficiency virus 2',
 'Sars related coronavirus',
 'Rhinovirus',
 'Influenza b virus',
 'Parainfluenza virus']

In [27]:
all_df["fre"] = 0
for k, pdf in pdf_d.items():
    
    all_df.loc[all_df["物种名称"].isin(list(pdf.loc[pdf["label_counts"] >= 1000].index)),"fre"] = ">=1000"
    
    all_df.loc[all_df["物种名称"].isin(list(pdf.loc[(pdf["label_counts"] >= 100) & (pdf["label_counts"]<1000)].index)),"fre"] = "100-999"
    
    all_df.loc[all_df["物种名称"].isin(list(pdf.loc[(pdf["label_counts"] >= 10) & (pdf["label_counts"]<100)].index)),"fre"] = "10-99"
    
    all_df.loc[all_df["物种名称"].isin(list(pdf.loc[(pdf["label_counts"] >= 1) & (pdf["label_counts"]<10)].index)),"fre"] = "1-9"

In [15]:
(np.sum(pdf_d["Bactology"]["label_counts"][0:11])+np.sum(pdf_d["Mycology"]["label_counts"][0:3])+np.sum(pdf_d["Virology"]["label_counts"][0:10]))/(np.sum(pdf_d["Bactology"]["label_counts"])+np.sum(pdf_d["Mycology"]["label_counts"])+np.sum(pdf_d["Virology"]["label_counts"]))

0.5820962375223957

In [13]:
for k, pdf in pdf_d.items():
    print(pdf.loc[pdf["label_counts"] >= 1000].shape[0],pdf.loc[pdf["label_counts"] >= 1000].shape[0]/pdf.shape[0])
    print(pdf.loc[(pdf["label_counts"] >= 100) & (pdf["label_counts"]<1000)].shape[0],pdf.loc[(pdf["label_counts"] >= 100) & (pdf["label_counts"]<1000)].shape[0]/pdf.shape[0])
    print(pdf.loc[(pdf["label_counts"] >= 10) & (pdf["label_counts"]<100)].shape[0],pdf.loc[(pdf["label_counts"] >= 10) & (pdf["label_counts"]<100)].shape[0]/pdf.shape[0])
    print(pdf.loc[(pdf["label_counts"] >= 1) & (pdf["label_counts"]<10)].shape[0],pdf.loc[(pdf["label_counts"] >= 1) & (pdf["label_counts"]<10)].shape[0]/pdf.shape[0])

11 0.0171606864274571
33 0.0514820592823713
139 0.21684867394695787
458 0.7145085803432137
3 0.010067114093959731
11 0.03691275167785235
61 0.20469798657718122
223 0.7483221476510067
10 0.020964360587002098
63 0.1320754716981132
151 0.3165618448637317
253 0.5303983228511531


# 患者信息

In [6]:
np.mean(all_df.loc[all_df["标注方式"]=="人工标注"]!="none")

/home/data/t070224/miniconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3438: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


物种名称           1.000000
本文中是肺部病原的概率    1.000000
样本量            0.909427
年龄             0.541590
患者性别           0.460259
免疫状态           0.256932
证据等级           0.432532
标注方式           1.000000
原文链接           1.000000
物种类别           1.000000
dtype: float64

In [62]:
np.mean(all_df!="none")

/home/data/t070224/miniconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3438: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


物种名称           1.000000
本文中是肺部病原的概率    1.000000
样本量            0.800011
年龄             0.685647
患者性别           0.534279
免疫状态           0.314737
证据等级           0.327663
标注方式           1.000000
原文链接           1.000000
物种类别           1.000000
dtype: float64

In [7]:
# 样本量
pdf_add = pd.DataFrame(all_df["样本量"].value_counts())
pdf_add = pdf_add.loc[['1-9','10-99','100以上']]

for spec_type in spec_paper_dics.keys():
    
    pdf = all_df.loc[(all_df["物种类别"]==spec_type)]
    sub_add = pd.DataFrame(pdf["样本量"].value_counts())
    sub_add = sub_add.loc[['1-9','10-99','100以上']]

    sub_add.columns = [spec_type]
    pdf_add = pd.concat((pdf_add,round(sub_add/np.sum(sub_add),4)*100),axis=1)
pdf_add = pdf_add[['Bactology', 'Mycology', 'Virology']].T
pdf_add

,1-9,10-99,100以上
Bactology,48.53,26.58,24.89
Mycology,62.86,24.03,13.11
Virology,33.54,28.38,38.09


In [8]:
# 性别
pdf_add = pd.DataFrame(all_df["患者性别"].value_counts())
pdf_add = pdf_add.loc[['男', '女']]

for spec_type in spec_paper_dics.keys():
    
    pdf = all_df.loc[(all_df["物种类别"]==spec_type)]
    sub_add = pd.DataFrame(pdf["患者性别"].value_counts())
    sub_add = sub_add.loc[['男', '女']]

    sub_add.columns = [spec_type]
    pdf_add = pd.concat((pdf_add,round(sub_add/np.sum(sub_add),4)*100),axis=1)
pdf_add = pdf_add[['Bactology', 'Mycology', 'Virology']].T
pdf_add

,男,女
Bactology,69.55,30.45
Mycology,70.90,29.10
Virology,71.63,28.37


In [9]:
# 年龄
pdf_add = pd.DataFrame(all_df["年龄"].value_counts())
pdf_add = pdf_add.loc[['1岁以下','1-17岁','18-64岁','65岁以上']]

for spec_type in spec_paper_dics.keys():
    
    pdf = all_df.loc[(all_df["物种类别"]==spec_type)]
    sub_add = pd.DataFrame(pdf["年龄"].value_counts())
    sub_add = sub_add.loc[['1岁以下','1-17岁','18-64岁','65岁以上']]

    sub_add.columns = [spec_type]
    pdf_add = pd.concat((pdf_add,round(sub_add/np.sum(sub_add),4)*100),axis=1)
pdf_add = pdf_add[['Bactology', 'Mycology', 'Virology']].T
pdf_add

,1岁以下,1-17岁,18-64岁,65岁以上
Bactology,4.32,26.74,45.95,22.99
Mycology,3.69,23.59,56.44,16.28
Virology,7.44,41.66,35.38,15.53


In [10]:
# 免疫状态
pdf_add = pd.DataFrame(all_df["免疫状态"].value_counts())
pdf_add = pdf_add.loc[['none','抑制']]

for spec_type in spec_paper_dics.keys():
    
    pdf = all_df.loc[(all_df["物种类别"]==spec_type)]
    sub_add = pd.DataFrame(pdf["免疫状态"].value_counts())
    sub_add.columns = [spec_type]
    pdf_add = pd.concat((pdf_add,round(sub_add/np.sum(sub_add),4)*100),axis=1)
pdf_add = pdf_add[['Bactology', 'Mycology', 'Virology']].T
pdf_add

,none,抑制
Bactology,71.74,28.26
Mycology,42.19,57.81
Virology,71.94,28.06


In [6]:
# 证据等级
pdf_add = pd.DataFrame(all_df["证据等级"].value_counts())
pdf_add = pdf_add.loc[[
                       '5：气管吸出物或痰/BALF纯培养或优势培养出病原体',
                       '4：气管镜活检组织培养出病原体，或者支气管肺泡灌洗液（BALF）定量培养>10**4', 
                       '3：血及无菌体液（如脓胸液）', 

                       '1：同时具备肺组织病理+病原学证据',
                       ]]

for spec_type in spec_paper_dics.keys():
    
    pdf = all_df.loc[(all_df["物种类别"]==spec_type)]
    sub_add = pd.DataFrame(pdf["证据等级"].value_counts())
    sub_add = sub_add.loc[[
                       '5：气管吸出物或痰/BALF纯培养或优势培养出病原体',
                       '4：气管镜活检组织培养出病原体，或者支气管肺泡灌洗液（BALF）定量培养>10**4', 
                       '3：血及无菌体液（如脓胸液）', 

                       '1：同时具备肺组织病理+病原学证据',
                       ]]
    sub_add.columns = [spec_type]
    pdf_add = pd.concat((pdf_add,round(sub_add/np.sum(sub_add),4)*100),axis=1)
    
pdf_add = pdf_add[['Bactology', 'Mycology', 'Virology']].T
pdf_add

,5：气管吸出物或痰/BALF纯培养或优势培养出病原体,4：气管镜活检组织培养出病原体，或者支气管肺泡灌洗液（BALF）定量培养>10**4,3：血及无菌体液（如脓胸液）,1：同时具备肺组织病理+病原学证据
Bactology,36.61,14.00,39.12,10.28
Mycology,16.30,26.71,29.07,27.92
Virology,33.26,20.63,29.55,16.56


# 免疫状态

In [12]:
pdf_nmd = {}

for spec_type in spec_paper_dics.keys():
    
    pdf = pd.DataFrame(all_df.loc[(all_df["物种类别"]==spec_type) & (all_df["免疫状态"]=="none"),"物种名称"].value_counts())
    pdf.columns = ["label_counts"]
    pdf["search_counts"] = 0
    pdf["portions"] = 0
    spdf = pd.DataFrame([spec_paper_dics[spec_type].keys(), [len(spec_paper_dics[spec_type][i]) for i in spec_paper_dics[spec_type].keys()]],index=["物种名称","Count"]).T
    spdf.index = spdf["物种名称"]
    
    for i in pdf.index:       
        pdf.loc[i, "search_counts"] = all_df["物种名称"].value_counts()[i]
        pdf.loc[i, "portions"] = (pdf.loc[i, "label_counts"]/pdf.loc[i, "search_counts"]).round(4)
    pdf.index = [i.capitalize() for i in pdf.index]
    pdf_nmd[spec_type] = pdf

In [13]:
pdf_md = {}

for spec_type in spec_paper_dics.keys():
    
    pdf = pd.DataFrame(all_df.loc[(all_df["物种类别"]==spec_type) & (all_df["免疫状态"]=="抑制"),"物种名称"].value_counts())
    pdf.columns = ["label_counts"]
    pdf["search_counts"] = 0
    pdf["portions"] = 0
    spdf = pd.DataFrame([spec_paper_dics[spec_type].keys(), [len(spec_paper_dics[spec_type][i]) for i in spec_paper_dics[spec_type].keys()]],index=["物种名称","Count"]).T
    spdf.index = spdf["物种名称"]
    
    for i in pdf.index:       
        pdf.loc[i, "search_counts"] = all_df["物种名称"].value_counts()[i]
        pdf.loc[i, "portions"] = (pdf.loc[i, "label_counts"]/pdf.loc[i, "search_counts"]).round(4)
    pdf.index = [i.capitalize() for i in pdf.index]
    pdf_md[spec_type] = pdf

In [20]:
pdf_md["Bactology"].loc[(pdf_md["Bactology"]["portions"] > 0.5)].sort_values("search_counts")[-14:]

,label_counts,search_counts,portions
Nocardia transvalensis,16,28,0.5714
Achromobacter denitrificans,31,37,0.8378
Burkholderia multivorans,34,39,0.8718
Nocardia otitidiscaviarum,21,40,0.5250
Nocardia nova,28,44,0.6364
Nocardia cyriacigeorgica,35,67,0.5224
Burkholderia cenocepacia,63,72,0.8750
Bordetella bronchiseptica,55,82,0.6707
Achromobacter xylosoxidans,71,92,0.7717
Nocardia farcinica,76,145,0.5241


In [19]:
a = pdf_nmd["Bactology"].loc[(pdf_nmd["Bactology"]["portions"] >= 0.5)].sort_values("search_counts")[-10:]
a["label_counts"] = a["search_counts"]-a["label_counts"]
a["portions"] = 1 - a["portions"]
a

,label_counts,search_counts,portions
Escherichia coli,248,1096,0.2263
Mycobacterium intracellulare,558,1380,0.4043
Legionella pneumophila,291,1387,0.2098
Klebsiella pneumoniae,387,1694,0.2285
Mycobacterium avium,848,1988,0.4266
Haemophilus influenzae,395,1988,0.1987
Mycoplasma pneumoniae,244,2986,0.0817
Staphylococcus aureus,834,3600,0.2317
Streptococcus pneumoniae,786,5051,0.1556
Mycobacterium tuberculosis,2338,7686,0.3042


In [30]:
for spec_type in pdf_d:
    pdfa= pdf_d[spec_type]
    pdf = pdf_md[spec_type].loc[(pdf_md[spec_type]["portions"] >= 0.5)]
    
    print(pdf.loc[pdf["search_counts"] >= 1000].shape[0],pdf.loc[pdf["search_counts"] >= 1000].shape[0]/pdfa.loc[pdfa["label_counts"] >= 1000].shape[0])
    
    print(
        pdf.loc[(pdf["search_counts"] >= 100) & (pdf["search_counts"]<1000)].shape[0],
        pdf.loc[(pdf["search_counts"] >= 100) & (pdf["search_counts"]<1000)].shape[0]/pdfa.loc[(pdfa["label_counts"] >= 100) & (pdfa["label_counts"]<1000)].shape[0])
    
    print(pdf.loc[(pdf["search_counts"] >= 10) & (pdf["search_counts"]<100)].shape[0],pdf.loc[(pdf["search_counts"] >= 10) & (pdf["search_counts"]<100)].shape[0]/pdfa.loc[(pdfa["label_counts"] >= 10) & (pdfa["label_counts"]<100)].shape[0])
    
    print(pdf.loc[(pdf["search_counts"] >= 1) & (pdf["search_counts"]<10)].shape[0],pdf.loc[(pdf["search_counts"] >= 1) & (pdf["search_counts"]<10)].shape[0]/pdfa.loc[(pdfa["label_counts"] >= 1) & (pdfa["label_counts"]<10)].shape[0])
    

1 0.09090909090909091
4 0.12121212121212122
24 0.17266187050359713
141 0.3078602620087336
2 0.6666666666666666
4 0.36363636363636365
38 0.6229508196721312
145 0.6502242152466368
3 0.3
6 0.09523809523809523
25 0.16556291390728478
63 0.2490118577075099


In [29]:
pdfa

,label_counts,search_counts,portions
Influenza a virus,4710,69747,0.0675
Respiratory syncytial virus,4628,16380,0.2825
Human immunodeficiency virus,3946,103361,0.0382
H1n1,1794,22454,0.0799
Human immunodeficiency virus 1,1681,62922,0.0267
...,...,...,...
Flanders virus,1,88,0.0114
Powassan virus,1,283,0.0035
Rhinovirus c3,1,5,0.2000
Vaccinia virus,1,15068,0.0001


# 分布频率与信息统计

In [ ]:
# 样本量

pdf_add = pd.DataFrame(all_df.loc[all_df["物种类别"]=="Bactology","样本量"].value_counts())
pdf_add = pdf_add.loc[['1-9','10-99','100以上']]

for spec_type in set(all_df["fre"]):
    
    pdf = all_df.loc[(all_df["fre"]==spec_type)]
    sub_add = pd.DataFrame(pdf["样本量"].value_counts())
    sub_add = sub_add.loc[['1-9','10-99','100以上']]

    sub_add.columns = [spec_type]
    pdf_add = pd.concat((pdf_add,round(sub_add/np.sum(sub_add),4)*100),axis=1)
pdf_add = pdf_add[['1-9', "10-99", "100-999",">=1000"]].T
pdf_add

In [68]:
# 性别
pdf_add = pd.DataFrame(all_df.loc[all_df["物种类别"]=="Bactology","患者性别"].value_counts())
pdf_add = pdf_add.loc[['男', '女']]

for spec_type in set(all_df["fre"]):
    
    pdf = all_df.loc[(all_df["fre"]==spec_type)]
    sub_add = pd.DataFrame(pdf["患者性别"].value_counts())
    sub_add = sub_add.loc[['男', '女']]

    sub_add.columns = [spec_type]
    pdf_add = pd.concat((pdf_add,round(sub_add/np.sum(sub_add),4)*100),axis=1)
pdf_add = pdf_add[['1-9', "10-99", "100-999",">=1000"]].T
pdf_add

,男,女
1-9,71.98,28.02
10-99,72.80,27.20
100-999,73.18,26.82
>=1000,68.73,31.27


In [71]:
# 年龄
pdf_add = pd.DataFrame(all_df.loc[all_df["物种类别"]=="Bactology","年龄"].value_counts())
pdf_add = pdf_add.loc[['1岁以下','1-17岁','18-64岁','65岁以上']]

for spec_type in set(all_df["fre"]):
    
    pdf = all_df.loc[(all_df["fre"]==spec_type)]
    sub_add = pd.DataFrame(pdf["年龄"].value_counts())
    sub_add = sub_add.loc[['1岁以下','1-17岁','18-64岁','65岁以上']]

    sub_add.columns = [spec_type]
    pdf_add = pd.concat((pdf_add,round(sub_add/np.sum(sub_add),4)*100),axis=1)
pdf_add = pdf_add[['1-9', "10-99", "100-999",">=1000"]].T
pdf_add

,1岁以下,1-17岁,18-64岁,65岁以上
1-9,4.96,26.10,48.61,20.33
10-99,5.31,32.82,43.48,18.38
100-999,6.73,39.18,36.75,17.34
>=1000,5.28,30.75,44.34,19.63


In [12]:
# 免疫状态
pdf_add = pd.DataFrame(all_df.loc[all_df["物种类别"]=="Bactology","免疫状态"].value_counts())
pdf_add = pdf_add.loc[['none','抑制']]

for spec_type in set(all_df["fre"]):
    
    pdf = all_df.loc[(all_df["fre"]==spec_type)]
    sub_add = pd.DataFrame(pdf["免疫状态"].value_counts())
    sub_add.columns = [spec_type]
    pdf_add = pd.concat((pdf_add,round(sub_add/np.sum(sub_add),4)*100),axis=1)
pdf_add = pdf_add[['1-9', "10-99", "100-999",">=1000"]].T
pdf_add

KeyError: "None of [Index(['1-9', '10-99', '100-999', '>=1000'], dtype='object')] are in the [columns]"

In [29]:
# 证据等级
pdf_add = pd.DataFrame(all_df["证据等级"].value_counts())
pdf_add = pdf_add.loc[[
                       '5：气管吸出物或痰/BALF纯培养或优势培养出病原体',
                       '4：气管镜活检组织培养出病原体，或者支气管肺泡灌洗液（BALF）定量培养>10**4', 
                       '3：血及无菌体液（如脓胸液）', 
                       '1：同时具备肺组织病理+病原学证据',
                       ]]

for spec_type in set(all_df["fre"]):
    
    pdf = all_df.loc[(all_df["fre"]==spec_type)]
    sub_add = pd.DataFrame(pdf["证据等级"].value_counts())
    sub_add = sub_add.loc[[
                       '5：气管吸出物或痰/BALF纯培养或优势培养出病原体',
                       '4：气管镜活检组织培养出病原体，或者支气管肺泡灌洗液（BALF）定量培养>10**4', 
                       '3：血及无菌体液（如脓胸液）', 
                       '1：同时具备肺组织病理+病原学证据',
                       ]]
    sub_add.columns = [spec_type]
    pdf_add = pd.concat((pdf_add,round(sub_add/np.sum(sub_add),4)*100),axis=1)
    
pdf_add = pdf_add[['1-9', "10-99", "100-999",">=1000"]].T
pdf_add

,5：气管吸出物或痰/BALF纯培养或优势培养出病原体,4：气管镜活检组织培养出病原体，或者支气管肺泡灌洗液（BALF）定量培养>10**4,3：血及无菌体液（如脓胸液）,1：同时具备肺组织病理+病原学证据
1-9,20.99,17.64,42.06,19.31
10-99,25.20,16.28,37.94,20.58
100-999,30.98,19.34,33.39,16.29
>=1000,35.65,17.97,33.50,12.88


In [11]:
pdf_add

,证据等级,0
5：气管吸出物或痰/BALF纯培养或优势培养出病原体,11676,32.59
4：气管镜活检组织培养出病原体，或者支气管肺泡灌洗液（BALF）定量培养>10**4,6502,18.15
3：血及无菌体液（如脓胸液）,12282,34.28
1：同时具备肺组织病理+病原学证据,5369,14.99
